In [1]:
import sqlite3
import random
from pyvis.network import Network
import networkx as nx
from functools import reduce

In [2]:
iOrigInfProb = 0.3
iVaccInfProb = 0
HexInfColor = '#fc0303'
# 第一個被感染的人
iFirstInfPid = 0

iDays = 2
iPopCnt = 500
iMaxConn = 10
listPR = []

In [3]:
class Person:
    def __init__(self, iPid, boolInfFlag=False, floatInfProbability=0, boolIfVacc=False):
        self.pid = iPid
        self.boolIfInf = boolInfFlag
        self.boolIfVacc = boolIfVacc
        self.floatInfProbability = floatInfProbability
        
    def IsInfected(self):
        self.boolIfInf = True
        self.floatInfProbability = iOrigInfProb
        
    def IsVacced(self):
        self.boolIfVacc = True
        self.boolIfInf = False
        self.floatInfProbability = iVaccInfProb
        
    def ResetInfStatus(self):
        self.boolIfInf = False
        self.floatInfProbability = 0
        self.boolIfVacc=False

In [4]:
connPR = sqlite3.connect('PeopleRelation.db')
cur = connPR.cursor()

In [5]:
# drop 所有表格
cur.execute('drop table if exists Relation')
cur.execute('drop table if exists People')

In [6]:
cur.execute(
'''
    create table if not exists People( pid integer not null primary key )
''')

cur.execute(
'''
    create table if not exists Relation( pid integer not null,
    pidconn integer not null,
    primary key(pid,
    pidconn),
    foreign key (pid) references People(pid),
    foreign key (pidconn) references People(pid) )
''')

In [7]:
listPeople = [Person(x) for x in range(iPopCnt)]

In [8]:
# insert data into table People
for person in listPeople:
    cur.execute(f'insert into People values({person.pid})')
connPR.commit()

In [9]:
# 模擬人與人之間的接觸關係
listPR = []
for person in listPeople:
    iRandconnCnt = random.randint(0, iMaxConn)
    listPidConnTo = random.choices(listPeople, k=iRandconnCnt)
    for PersonConnTo in listPidConnTo:
        listPR.append((person.pid, PersonConnTo.pid))
listPR = list(set(listPR))

In [10]:
cur.executemany('insert into Relation values(?, ?)', listPR)
connPR.commit()

In [11]:
netPR = Network(height='750px', width='100%', bgcolor='#222222', font_color='white')

for person in listPeople:
    netPR.add_node(person.pid, title=str(person.pid), value=10)

for relation in listPR:
    netPR.add_edge(relation[0], relation[1])

netPR.barnes_hut()
netPR.show('mygraph.html')

In [12]:
netPRInf = Network(height='750px', width='100%', bgcolor='#222222', font_color='white')

listPeople[iFirstInfPid].IsInfected()

for day in range(iDays):
    for person in listPeople:
        listRelationsWithThisPerson = list(filter(lambda x: x[0] == person.pid or x[1] == person.pid, listPR))
        # 挑出跟這個人有關的人際關係
        listMightBeInfPids = list(set(reduce(lambda x, y: x + y, listRelationsWithThisPerson)))
        # 移除這個人
        listMightBeInfPids.remove(person.pid)
        # 如果感染機率大於 0 ，就開始感染過程。
        if person.floatInfProbability > 0:
            # 根據這個人的感染機率隨機挑人
            listBeInfPids = random.choices(listMightBeInfPids, k=round(len(listMightBeInfPids) * person.floatInfProbability))
            for iInfPid in listBeInfPids:
                listPeople[iInfPid].IsInfected()

                    
for person in listPeople:
    if person.boolIfInf == True:
        netPRInf.add_node(person.pid, title=str(person.pid), color=HexInfColor, value=10)
    else:
        netPRInf.add_node(person.pid, title=str(person.pid), value=10)

for relation in listPR:
    netPRInf.add_edge(relation[0], relation[1])

netPRInf.barnes_hut()
netPRInf.show('InfGraph.html')